# Imports

In [1]:
import re
import numpy as np
import pandas as pd

# Cleaning dataframes

In [2]:
df = pd.read_csv('../data/df_houses_full_raw.csv')

In [3]:
# See what wee got.
df.head()

,Unnamed: 0,house_id,city,street,montly_rent,deposit_value,house_rules,size,lat_long,scrapy_datetime,lat,long
0,0,1719697,Berlin,Nürnberger Straße,EUR 850,EUR 1700,"['Private living room', 'Private Toilet', 'Pri...",Property: 22 m2,"['52.50027', '13.33596']",2021/05/13 - 12:16:03,52.50027,13.33596
1,1,1716752,Berlin,Schnellerstraße,EUR 450,EUR 900,"['Shared toilet', 'Shared kitchen', 'Unisex ba...",Property: 90 m2,"['52.45495', '13.51636']",2021/05/13 - 12:16:05,52.45495,13.51636
2,2,1716755,Berlin,Schnellerstraße,EUR 475,EUR 950,"['Shared toilet', 'Shared kitchen', 'Unisex ba...",Property: 90 m2,"['52.45495', '13.51636']",2021/05/13 - 12:16:07,52.45495,13.51636
3,3,1724139,Berlin,Otto-Braun-Straße,EUR 760,EUR 500,"['Private facilities', 'Private Toilet', 'Priv...",Property: 25 m2,"['52.52449', '13.41863']",2021/05/13 - 12:16:09,52.52449,13.41863
4,4,1721869,Berlin,Otto-Braun-Straße,EUR 760,EUR 500,"['Private facilities', 'Private Toilet', 'Priv...",Property: 25 m2,"['52.52449', '13.41863']",2021/05/13 - 12:16:11,52.52449,13.41863


In [4]:
df.shape

(1715, 12)

In [5]:
# Look for NaN and clean it.
df.isna().sum()

Unnamed: 0         0
house_id           0
city               0
street             0
montly_rent        0
deposit_value      0
house_rules        0
size               0
lat_long           0
scrapy_datetime    0
lat                0
long               0
dtype: int64

In [6]:
df.dtypes

Unnamed: 0           int64
house_id             int64
city                object
street              object
montly_rent         object
deposit_value       object
house_rules         object
size                object
lat_long            object
scrapy_datetime     object
lat                float64
long               float64
dtype: object

## Cleaning df_houses

In [7]:
# Drop "Unnamed: 0" column and 'lat_long' column
df.drop(columns = ['Unnamed: 0', 'lat_long'], inplace=True)

# Clean "montly_rent"
df['montly_rent'] = df['montly_rent'].apply(lambda x: re.search('\d+', x).group(0))
df['montly_rent'] = df['montly_rent'].astype(int)

# Clean 'deposit_value'
df['deposit_value'] = df['deposit_value'].apply(lambda x: re.search('\d+', x).group(0))
df['deposit_value'] = df['deposit_value'].astype(int)

# Clean 'house_rules'
## we are looking for places that pets are allowed, so we'll filter the 'house_rules' column to find this places.
yes = 'Pets allowed'
maybe = 'Pets negotiable'
no = 'Pets not allowed'

df['pets'] = df['house_rules'].apply(lambda x: yes if yes in x else maybe if maybe in x else no if no in x else np.nan)

# Clean 'size'
## There are some 'Furnished' as size. I'll exclude them.
df = df[df['size'] != 'Furnished']
df['size'] = df['size'].apply(lambda x: x.split(' ')[1])
df['size'] = df['size'].astype(float)

# Create 'm2 value'.
df['m2_value'] = df['montly_rent'] / df['size']

# Clean columns
df.drop(columns = ['house_rules'], inplace = True)

# Reorder columns
df = df[['house_id', 'scrapy_datetime',	'city',	'street', 'size', 'pets', 'montly_rent', 'deposit_value', 'm2_value', 'lat', 'long']]

# scrape datetime - to datetime
df['scrapy_datetime'] = pd.to_datetime(df['scrapy_datetime'])

In [8]:
df.head()

,house_id,scrapy_datetime,city,street,size,pets,montly_rent,deposit_value,m2_value,lat,long
0,1719697,2021-05-13 12:16:03,Berlin,Nürnberger Straße,22.0,Pets not allowed,850,1700,38.636364,52.50027,13.33596
1,1716752,2021-05-13 12:16:05,Berlin,Schnellerstraße,90.0,Pets not allowed,450,900,5.000000,52.45495,13.51636
2,1716755,2021-05-13 12:16:07,Berlin,Schnellerstraße,90.0,Pets not allowed,475,950,5.277778,52.45495,13.51636
3,1724139,2021-05-13 12:16:09,Berlin,Otto-Braun-Straße,25.0,Pets not allowed,760,500,30.400000,52.52449,13.41863
4,1721869,2021-05-13 12:16:11,Berlin,Otto-Braun-Straße,25.0,Pets not allowed,760,500,30.400000,52.52449,13.41863


In [9]:
df.isna().sum()

house_id            0
scrapy_datetime     0
city                0
street              0
size                0
pets               33
montly_rent         0
deposit_value       0
m2_value            0
lat                 0
long                0
dtype: int64

In [11]:
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)

In [12]:
df.to_csv('../data/df_houses_full_cleanned.csv', index = False)

## Cleanning nearby_venues dataset

In [13]:
df_venues = pd.read_csv('../data/nearby_venues_full_raw.csv')

In [14]:
df_venues.head()

,house_id,house_latitude,hoouse_longitude,venue,city,venue_category,venue_latitude,venue_longitude
0,1719697,52.50027,13.33596,Salut Mediterranean Food & Catering,"['Augsburger Str 29', '10789 Berlin', 'Deutsch...",Mediterranean Restaurant,52.501707,13.334932
1,1719697,52.50027,13.33596,DORMERO Hotel Berlin Ku’damm,"['Eislebener Str. 14', '10789 Berlin', 'Deutsc...",Hotel,52.500549,13.333723
2,1719697,52.50027,13.33596,Duke,"['Nürnberger Str. 50-55', '10789 Berlin', 'Deu...",New American Restaurant,52.501667,13.338187
3,1719697,52.50027,13.33596,City Imbiss,"['Lietzenburger Straße 29', '10789 Berlin', 'D...",Turkish Restaurant,52.499093,13.335612
4,1719697,52.50027,13.33596,Lego Store,"['Tauentzienstraße 20', '10789 Berlin', 'Deuts...",Toy / Game Store,52.502534,13.339502


In [15]:
# Adress
df_venues['address'] = df_venues['city'].apply(lambda x: x.split(',')[0][2:-2])

# city
df_venues['city'] = df_venues['city'].apply(lambda x: x.split(',')[:-1][1:])

extract = []

# get empty values as NaN
for x in df_venues.index:
    if len(df_venues.iloc[x][4]) == 0:
        extract.append(np.nan)
    else:
        extract.append(df_venues.iloc[x][4])

# assign extract list to "city" column - with NaN
df_venues['city'] = extract

# drop NaN
df_venues.dropna(inplace = True)

# get only cities names
df_venues['city'] = df_venues['city'].apply(lambda x: x[0].split(' ')[-1])

# reorder columns
df_venues = df_venues[['house_id', 'city', 'venue', 'address',	'venue_category', 'venue_latitude',	'venue_longitude']]

In [16]:
df_venues.drop_duplicates(inplace = True)

In [17]:
df_venues.head()

,house_id,city,venue,address,venue_category,venue_latitude,venue_longitude
0,1719697,Berlin',Salut Mediterranean Food & Catering,Augsburger Str 2,Mediterranean Restaurant,52.501707,13.334932
1,1719697,Berlin',DORMERO Hotel Berlin Ku’damm,Eislebener Str. 1,Hotel,52.500549,13.333723
2,1719697,Berlin',Duke,Nürnberger Str. 50-5,New American Restaurant,52.501667,13.338187
3,1719697,Berlin',City Imbiss,Lietzenburger Straße 2,Turkish Restaurant,52.499093,13.335612
4,1719697,Berlin',Lego Store,Tauentzienstraße 2,Toy / Game Store,52.502534,13.339502


In [18]:
df_venues.to_csv('../data/nearby_venues_full_cleanned.csv', index = False)